``` PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res:  <http://dbpedia.org/resource/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT DISTINCT ?uri, ?name, ?birth, ?position, ?number
WHERE { 
        {?uri dbo:team res:Boston_Celtics} UNION {?uri dbo:draftTeam res:Boston_Celtics}
        ?uri foaf:name ?name .
        ?uri dbo:birthDate ?birth .
        FILTER (regex (?birth, '\\d')) .
        ?uri dbo:position ?position .
        ?uri dbo:number ?number
} 
```

In [7]:
import pandas as pd
import wikipedia as w
import spacy
import re
from datetime import datetime
import calendar
# nlp = spacy.load("en_core_web_md")
# nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [3]:
df = pd.read_csv('/Users/liudmylaslava/Downloads/sparql.csv')
df.head()

,uri,name,birth,position,number
0,http://dbpedia.org/resource/James_Young_(baske...,James Young,1995-8-16,http://dbpedia.org/resource/Shooting_guard,13
1,http://dbpedia.org/resource/James_Young_(baske...,James Young,1995-8-16,http://dbpedia.org/resource/Small_forward,13
2,http://dbpedia.org/resource/Isaiah_Thomas_(bas...,Isaiah Thomas,1989-2-7,http://dbpedia.org/resource/Point_guard,4
3,http://dbpedia.org/resource/Jae_Crowder,Jae Crowder,1990-7-6,http://dbpedia.org/resource/Small_forward,99
4,http://dbpedia.org/resource/Jonas_Jerebko,Jonas Jerebko,1987-3-2,http://dbpedia.org/resource/Forward_(basketball),8


In [154]:
# pd.to_datetime(df['birth'])

0     1995-08-16
1     1995-08-16
2     1989-02-07
3     1990-07-06
4     1987-03-02
         ...    
141   1932-04-26
142   1923-03-10
143   1923-03-10
144   1923-03-10
145   1923-03-10
Name: birth, Length: 146, dtype: datetime64[ns]

In [4]:
li = list(set(df['uri'].tolist()))
ents = [l.split('/')[-1].replace('_', ' ') for l in li]
text=[]
titles = []
for en in ents:
    try:
        page = w.page(en+' basketball')
        text.append(page.content)
        titles.append(page.title)
    except Exception:
        pass

In [5]:
with open('text.txt', 'w') as f:
    f.writelines(text)

In [14]:
positions = [ 'point guard' , 'shooting guard', 'small forward', 'power forward', 'center']
data = []
for t in text:
    name, birth, position, number = 0,0,0,0
    doc = nlp(t)
    sentences = [sent.string.strip() for sent in doc.sents]
    first = ''.join(sentences[:2])
    birth = first[first.find("(")+1:first.find(")")]
    name = first[0:first.find('(')]
    po = []
    for pos in positions:
        if pos in t:
            po.append(pos)
            position = ','.join(po)
    num = []
    for n in [m.start() for m in re.finditer('No', t)]:
        if t[n+2] == '.':
            res = t[n+3:n+6]
            num.append(res)
            number = ','.join(num)
                 
    data.append([name, birth, position, number])

In [16]:
raw = pd.DataFrame(data = data, columns = ['name', 'birth', 'position', 'number'])
raw.head()

,name,birth,position,number
0,Rade Zagorac,"Serbian Cyrillic: Раде Загорац, born 12 August...",small forward,0
1,Jeff Reed Judkins,"born March 23, 1956",shooting guard,0
2,Cyrus Mann Jr.,"born April 2, 1956",center,0
3,George A. Kaftan,"February 22, 1928 – October 6, 2018",center,0
4,John J. Ezersky,"March 21, 1922 – February 20, 2012",0,0


In [22]:
m ={}
for i in range(1,13):
    m[i] = calendar.month_name[i]
list(m.keys())[list(m.values()).index('December')]
raw = pd.DataFrame(data = data, columns = ['name', 'birth', 'position', 'number'])
raw['birth'] = raw['birth'].apply(lambda x: x.split('–')[0].replace(',',''))
raw['birth'] = raw['birth'].apply(lambda x: ', '.join([t.text for t in nlp(x) if t.ent_type_=='DATE']))
raw['year']= raw['birth'].str[-4:]
raw['month'] = raw['birth'].apply(lambda x: ''.join([t.text for t in nlp(x) if t.pos_=='PROPN']))
raw['bir'] = raw['birth'].str[:-6]
raw['day'] = raw['bir'].apply(lambda x: ''.join([t.text for t in nlp(x) if t.pos_=='NUM']))
raw['month'] = raw[raw['month']!='']['month'].apply(lambda x: list(m.keys())[list(m.values()).index(x)])
raw['month'] = raw['month'].astype(str).str[:1]
raw['birth'] = raw['year']+'-'+raw['month']+'-'+raw['day']
raw.drop(['year','month','bir','day'], axis=1, inplace=True)
raw = raw[raw['birth']!='-n-']
raw.head()

,name,birth,position,number
0,Rade Zagorac,1995-8-12,small forward,0
1,Jeff Reed Judkins,1956-3-23,shooting guard,0
2,Cyrus Mann Jr.,1956-4-2,center,0
3,George A. Kaftan,1928-2-22,center,0
4,John J. Ezersky,1922-3-21,0,0


In [44]:
res = pd.merge(raw, df, on='birth')
res = res[['name_x','name_y']].drop_duplicates()
res.head()

,name_x,name_y
0,Rade Zagorac,Rade Zagorac
2,Jeff Reed Judkins,Jeff Judkins
4,Cyrus Mann Jr.,Cyrus Mann
5,George A. Kaftan,George Kaftan
6,John J. Ezersky,Johnny Ezersky


In [46]:
def f(x):
    res = any(z == y for z in x[0].split() for y in x[1].split())
    return res
res['match'] = res.apply(f, axis=1)
res.head()

,name_x,name_y,match
0,Rade Zagorac,Rade Zagorac,True
2,Jeff Reed Judkins,Jeff Judkins,True
4,Cyrus Mann Jr.,Cyrus Mann,True
5,George A. Kaftan,George Kaftan,True
6,John J. Ezersky,Johnny Ezersky,True


In [47]:
res['match'].value_counts()

True     87
False     2
Name: match, dtype: int64